<div style="padding: 10px; margin-bottom: 10px; border: 1px solid #333333;">
    
## Support Mission 9 ［モデルによる判定結果に基づくロボットカーの制御］
- このSupport Missionでは，Support Mission 6で作成したモデルに基づいて，カメラに写っている状況がfreeである確率と<br>
  blockedである確率を取得し，その確率を使ってロボットの制御を行います．
    
#### 完了の定義
- JetBotが赤いレゴブロックを検知したときに停止する．
- JetBotが赤いレゴブロックを検知していないときに前進する．

## 再利用タグ：[ロボットの制御]，[モデルによる撮影画像の判定]，[画像表示]

In [1]:
from jetbot import Robot
import time

C_ROBOT = Robot()

In [2]:
import torch
import torchvision

# 学習済みモデルを初期化しています
C_MODEL = torchvision.models.alexnet(pretrained=False)
C_MODEL.classifier[6] = torch.nn.Linear(C_MODEL.classifier[6].in_features, 2)

In [3]:
C_MODEL.load_state_dict(torch.load('best_model.pth'))

In [4]:
device = torch.device('cuda')
C_MODEL = C_MODEL.to(device)

In [5]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

# カメラからの画像を正規化するメソッドを定義しています
def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [6]:
import traitlets
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

# カメラからの映像の表示サイズを設定
C_IMAGE = widgets.Image(format='jpeg', width=224, height=224)

# 撮影される画像サイズの設定
C_CAMERA = Camera.instance(width=224, height=224)
C_CAMERA_LINK = traitlets.dlink((C_CAMERA, 'value'), (C_IMAGE, 'value'), transform=bgr8_to_jpeg)

# カメラからの映像を表示
display(C_IMAGE)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

## <span style="color: red; ">NEW</span> タグ：[モデルによる判定結果に基づくロボットカーの制御]

In [7]:
import torch.nn.functional as F
import time

def update(change):
    global C_ROBOT
    x = change['new'] 
    x = preprocess(x)
    y = C_MODEL(x)
        
    y = F.softmax(y, dim=1)

    # 重要！
    # 下記のプログラムでblockedの確率を取得している
    # datasetディレクトリ内では上がblockedディレクトリ,下がfreeディレクトリになっている
    # つまり，blockedの確率を取得したい場合[0]番目にその値が入っており，freeの確率を取得したい場合[1]番目その値が入っている
    # blockedよりも上にディレクトリが作成された場合は，そのディレクトリの確率が[0]に入っていることになる
    # 確率prob_blockedは 0 < prob_blocked <= 1 である
    prob_blocked = float(y.flatten()[0])
    
    if prob_blocked > 0.5:
        C_ROBOT.stop()
    else:
        C_ROBOT.forward(speed = 0.3)

    time.sleep(0.001)
    
update({'new': C_CAMERA.value}) 

In [8]:
C_CAMERA.observe(update, names='value') 

- ロボットを停止させます．

In [9]:
C_CAMERA.unobserve(update, names='value')
C_ROBOT.stop()

- カメラを切断します．

In [10]:
C_CAMERA_LINK.unlink()  # don't stream to browser (will still run camera)

In [11]:
C_CAMERA_LINK.link()  # stream to browser (wont run camera)